## Jupyter Notebook to test Azure ML Registry capabilities, with model sharing across Azure ML workspaces

### Environment setup

In [1]:
# Import required packages
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model, ManagedOnlineEndpoint, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential, AzureCliCredential, InteractiveBrowserCredential

In [ ]:
# Set required variables
subscription_id = "<your-subscription-id>" # Replace with your subscription ID

aml_staging_name = "<your-aml-staging-name>" # Replace with your AML staging workspace name
aml_staging_rg = "<your-aml-staging-rg>" # Replace with your AML staging resource group name

aml_production_name = "<your-aml-production-name>" # Replace with your AML production workspace name
aml_production_rg = "<your-aml-production-rg>" # Replace with your AML production resource group name

aml_registry_name = "<your-aml-registry-name>" # Replace with your AML registry name
aml_registry_location = "<your-aml-registry-location>" # Replace with your AML registry location

model_name = "<your-model-name>" # Replace with your model name
model_path = "<your-model-path>" # Replace with your model path
model_version = "<your-model-version>" # Replace with your model version

In [3]:
# Authenticate to Azure
try:
    credential = AzureCliCredential()
    # credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

In [ ]:
# Connect Staging Azure ML workspace
ml_client_staging = MLClient(
    credential = credential,
    subscription_id = subscription_id,
    resource_group_name = aml_staging_rg,
    workspace_name = aml_staging_name
)

print(ml_client_staging)

In [ ]:
# Connect Production Azure ML workspace
ml_client_prod = MLClient(
    credential = credential,
    subscription_id = subscription_id,
    resource_group_name = aml_production_rg,
    workspace_name = aml_production_name
)

print(ml_client_prod)

In [ ]:
# Connect Azure ML Registry
ml_client_registry = MLClient(
    credential = credential,
    registry_name = aml_registry_name,
    registry_location = aml_registry_location
)

print(ml_client_registry)

### Scenario 1: Creating model directly in registry

*Attempts to register model with the same name and version will fail.*

In [ ]:
# Register model directly in registry
mlflow_model = Model(
    path = model_path,
    type = AssetTypes.MLFLOW_MODEL,
    name = model_name + "-direct",
    version = model_version,
    description = "MLflow model, registered directly in the registry",
)

ml_client_registry.models.create_or_update(mlflow_model)

In [8]:
# Workaround option for AML Registry error
def model_register(client:MLClient):
    previous_model = client.models.get(
        name = mlflow_model.name,
        label = "latest"
    )
    mlflow_model.name = previous_model.name
    mlflow_model.version = previous_model.version
    client.models.create_or_update(mlflow_model)

model_register(ml_client_registry)

### Scenario 2: Sharing model via registry

In [ ]:
# Create model in Staging workspace
model_staging = Model(
    path = model_path,
    type = AssetTypes.MLFLOW_MODEL,
    name = model_name + "-staging",
    version = model_version,
    description = "MLflow model, registered in Staging workspace",
)

ml_client_staging.models.create_or_update(model_staging)

In [ ]:
# Share model from Staging to Registry
ml_client_staging.models.share(
    name = model_name + "-staging",
    version = model_version,
    registry_name = aml_registry_name,
    share_with_name = model_name + "-shared",
    share_with_version = model_version
)

In [11]:
# Create online in Production workspace
online_endpoint_name = "endpoint-shared-model"
endpoint = ManagedOnlineEndpoint(
    name = online_endpoint_name,
    description = "Online endpoint for shared model",
    auth_mode = "key"
)

ml_client_prod.begin_create_or_update(endpoint)

In [12]:
# Retrieve model from registry
shared_model = ml_client_registry.models.get(
    name = model_name + "-shared",
    version = model_version
)

In [13]:
# Deploy model in Production workspace
model_deployment = ManagedOnlineDeployment(
    name = model_name + "-shared",
    endpoint_name = online_endpoint_name,
    model = shared_model,
    instance_type = "Standard_F4s_v2",
    instance_count = 1
)

ml_client_prod.online_deployments.begin_create_or_update(model_deployment)

Check: endpoint endpoint-shared-model exists


...............................................................................

In [14]:
# Enable traffic for the endpoint
endpoint.traffic = {model_name + "-shared": 100}

ml_client_prod.begin_create_or_update(endpoint)